In [28]:
import requests
import os
import json
import glob
import ast

In [4]:
g_core_name="honzuki"
g_url_base="http://127.0.0.1:8983/solr/"+g_core_name+"/"

In [5]:
def get_url_add(url_base,is_commit=True):
    commit = "true" if is_commit else "false"
    header={"Content-Type": "text/json"}
    return url_base+"update?commit="+commit,header

In [6]:
add_data = {"id":"AAA","main_text":"test"}


In [8]:

url,header = get_url_add(g_url_base)
url

'http://127.0.0.1:8983/solr/honzuki/update?commit=true'

In [9]:
url,header = get_url_add(g_url_base)
print(url)
ret=requests.post(url,headers=header,data=json.dumps([add_data]))
print(ret)
print(ret.text)

http://127.0.0.1:8983/solr/honzuki/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":79}}



In [ ]:
file_list = glob.glob("C:/Users/mopper/SkyDrive/ドキュメント/本好きの下剋上/本好きの下克上まとめ/*/*.txt")
file_list

In [12]:
honzuki_url="https://ncode.syosetu.com/n4830bu/"
hanne_url="https://ncode.syosetu.com/n7835cj/"

In [15]:
def get_add_json(file_path,base_url):
    base_name = os.path.basename(file_path)
    dir_name = os.path.basename(os.path.dirname(file_path))
    
    base_name=base_name[:-4]
    book_no = base_name.split("_")[0]
    
    text=""
    
    with open(file_path,encoding="utf_8") as f:
        while(True):
            s=f.readline()
            if not s:
                break
            text+= s
            text+="\\n"
         
    url =base_url
    if url[-1]!="/":
        url+="/"
    url=url+book_no+"/"
    ret_dict ={
        "id":dir_name+"_"+book_no,
        "book_no":book_no,
        "chapter":dir_name,
        "title":base_name,
        "narou_url":url,
        "main_text":text}
    return ret_dict

In [16]:
url,header = get_url_add(g_url_base)
add_data = get_add_json(file_list[0],hanne_url)
print(url)
ret=requests.post(url,headers=header,data=json.dumps([add_data]))
print(ret)
print(ret.text)

http://127.0.0.1:8983/solr/honzuki/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":111}}



In [ ]:

for file_path in file_list:
    base_url="https://ncode.syosetu.com/n4830bu/"
    if "SS_ハンネローレ視点" in file_path:
        base_url="https://ncode.syosetu.com/n7835cj/"
    url,header = get_url_add(g_url_base)
    add_data = get_add_json(file_path,hanne_url)
    print(url)
    for key in add_data.keys():
        if key=="main_text":
            continue
        print("{0}:{1}".format(key,add_data[key][:100]))
    ret=requests.post(url,headers=header,data=json.dumps([add_data]))
    print(ret)
    print("---------------------")

In [26]:
url="http://127.0.0.1:8983/solr/honzuki/select?q=*%3A*"

ret=requests.post(url,headers=header)
print(ret)
print(ret.text[:100])

<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":0,
    "params":{
      "q":"*:*",
      "json":"


In [37]:
type(ret.text)

str

In [44]:
jsonData = ret.json()
jsonData.keys()

dict_keys(['responseHeader', 'response'])

In [47]:
jsonData = ret.json()
jsonData["response"].keys()

dict_keys(['numFound', 'start', 'numFoundExact', 'docs'])

In [50]:
jsonData = ret.json()
type(jsonData["response"]["docs"])

list

In [51]:
jsonData["response"]["docs"][0]

{'id': 'dummy_100',
 'book_no': '100',
 'chapter': 'dummy',
 'title': '02_新しい生活',
 'narou_url': 'dummy',
 'main_text': '第一部\u3000兵士の娘 dummy',
 '_version_': 1685852314957512704}

In [ ]:
search_word="フリーダ"

url=g_url_base+"select?q.alt="+search_word+"&wt=json&indent=true&defType=dismax&qf=main_text+title+chapter"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
for i in range(len(jsonData)):
    for key in jsonData[i].keys():
        data = jsonData[i][key]
        if type(data)==str:
            data=data[:50]
        print("{0}:{1}".format(key,data))
    print("-----------------------------")
   

In [ ]:
search_word="ダンケルフェルガー"

url=g_url_base+"select?q=\""+search_word+"\"&wt=json&indent=true&defType=dismax&qf=main_text+title+chapter&start=0&rows=50"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
print("ヒット数(上限50):",len(jsonData))
for i in range(len(jsonData)):
    print("{0}:[{1}]".format("title",jsonData[i]["title"]))
print("-----------------------------")
   

In [ ]:
search_word="ローゼマイン"

url=g_url_base+"select?q=\""+search_word+"\"&wt=json&indent=true&defType=dismax&qf=main_text+title+chapter&start=0&rows=50"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
for i in range(len(jsonData)):
    for key in jsonData[i].keys():
        if key=="_version_":
            continue
        data = jsonData[i][key]
        if type(data)==str:
            data=data[:50].replace("\n","")
        print("{0}:{1}".format(key,data))
    print("-----------------------------")
   